In [13]:
import numpy as np
import pandas as pd
import nltk
import os
import re
import warnings
from collections import defaultdict
from nltk.corpus import wordnet as wn, stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import io


In [2]:
doc_content = []  # all the content in the document
doc_name = []  # name of the document
files_path = []  # path to the documents
lexical_chain = []  # list of lexical chains from each document
total_features = []  # total number of features. 1652
final_training_Features = [] # to store features for training
corpus = [] # to store all text
doc_list_sequence = [] # store sequence of document read
actual_labels = {} # to store actual cluster of documents

In [3]:
def ReadDocuments(dir_name):
    current_file_id = 0
    current_dir_id = 0
    for Path in os.listdir(dir_name):
        sub_dir_path = os.path.join(dir_name, Path)
        for sub_dir_files in os.listdir(sub_dir_path):
            file_p = os.path.join(sub_dir_path, sub_dir_files)
            with open(file_p, "r") as file:
                FileContents = file.read()
                doc_content.append(FileContents.lower())
                doc_name.append(current_file_id)
                actual_labels[current_file_id] = current_dir_id 
                current_file_id+=1
                files_path.append(file_p)
        current_dir_id+=1

In [4]:
def in_wordnet(word):
    synsets = wn.synsets(word)
    return len(synsets) > 0

def contains_number(word):
    for char in word:
        if char.isnumeric():
            return True
    return False

def min_length_word(word):
    if  len(word) in [1,2]:
        return True
    return False

def custom_preprocessor(text):
    lematizer = WordNetLemmatizer()
    used_terms = {} # keep track of which terms have already been considered
    tokens = word_tokenize(text)
    filtered_tokens = []
    for word in tokens:
        if (not contains_number(word)) and (not min_length_word(word)) and (word not in stopwords.words('english')) and (in_wordnet(word)):
            lema_word = lematizer.lemmatize(word)
            if lema_word in used_terms.keys():
                continue
            else:
                used_terms[lema_word] = 0
                filtered_tokens.append(lema_word)
    return ' '.join(filtered_tokens)

In [5]:
ReadDocuments(os.getcwd() + "\BBC")
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', preprocessor=custom_preprocessor)
print("Building Features...")
X = vectorizer.fit_transform(doc_content)

# # save tdidf-features
# print("Saving tf-idf features")
# output_file_path = "tfidf_output.txt"
# with open(output_file_path, 'w', encoding='utf-8') as output_file:
#     np.savetxt(output_file, X.toarray(), fmt='%.4f')

Building Features...


c:\Users\umair\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  warnings.warn(


In [7]:
output_file_path = "bbc.txt"
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    np.savetxt(output_file, X.toarray(), fmt='%.4f')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
import os

# Read the text file
file_path = "input_file.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    documents = file.readlines()


# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the documents
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)


# Write TF-IDF results to a new file
output_file_path = "bbc.txt"
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    np.savetxt(output_file, tfidf_matrix.toarray(), fmt='%.4f')


In [23]:
file_name = os.getcwd() + "output.txt"
 
# Open the file in write mode
with open(file_name, "w") as file:
    # Iterate over the dictionary items and write them to the file
    for key, value in actual_labels.items():
        file.write(f"{value}\n")

print("Data written to", file_name)

Data written to c:\Users\umair\Desktop\FYP23-Deep-Document-Clustering\Base Linesoutput.txt


In [6]:
X

<2225x16821 sparse matrix of type '<class 'numpy.float64'>'
	with 266246 stored elements in Compressed Sparse Row format>

In [11]:
file_path = os.getcwd() + '/cite.txt'
def get_text_file_shape(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        num_lines = len(lines)
        max_line_length = max(len(line) for line in lines)
    return num_lines, max_line_length

# num_lines, max_line_length = get_text_file_shape(file_path)
# print("Number of lines:", num_lines)
# print("Maximum line length:", max_line_length)

In [12]:
file_path = os.getcwd() + '/bbc.terms'  # Replace "your_file_path_here.term" with the actual file path

try:
    with open(file_path, 'r') as file:
        content = file.read()
except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except Exception as e:
    print("An error occurred:", e)

num_lines, max_line_length = get_text_file_shape(file_path)
print("Number of lines:", num_lines)
print("Maximum line length:", max_line_length)

Number of lines: 9635
Maximum line length: 25


In [17]:
from sklearn.feature_extraction.text import TfidfTransformer

# Load the Matrix Market file
matrix = io.mmread("bbc.mtx")  # Replace "your_file.mtx" with the actual file path

# Initialize TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Fit and transform the data
tfidf_matrix = tfidf_transformer.fit_transform(matrix)

# Print the shape of the TF-IDF matrix
print("TF-IDF Matrix Shape:", tfidf_matrix.shape)

output_file_path = "bbc.txt"
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    np.savetxt(output_file, tfidf_matrix.toarray(), fmt='%.4f')


TF-IDF Matrix Shape: (9635, 2225)
